In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [28]:
raw_df = pd.read_csv('export_2017.csv', encoding = "utf-8")#, nrows = 100000)

In [29]:
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3


In [30]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 3.9+ GB


In [31]:
## should we have any missing customer data
cleaned_retail = raw_df.loc[pd.isnull(raw_df.CUS_N_KEY_CUSTOMER) == False]

In [32]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23768830 entries, 0 to 23768829
Data columns (total 22 columns):
SCM_N_ORDER_POS_NR         int64
SCM_N_ORDER_NR             int64
SCM_N_ORDER_NR_NC          int64
TDT_T_KEY_ORDER_DATE       object
ART_N_KEY_ARTICLE          int64
PPR_N_KEY_PHYS_PRODUCT     int64
CUS_N_KEY_CUSTOMER         int64
SCM_D_QUANTITY             float64
PPR_N_PHY_ARTICLE_ID       int64
PPR_N_PHY_PRODUCT_ID       int64
ART_N_KEY_ARTICLE_1        int64
ART_C_ART_NUMBER           float64
ART_C_PROD_NUMBER          int64
ART_V_ART_DESCRIPTION      object
ART_V_ART_DESCRIPTION_1    object
ART_V_PG_LEVEL3_DESC       object
ART_V_PG_LEVEL4_DESC       object
ART_V_PG_LEVEL5_DESC       object
ART_V_PG_LEVEL6_DESC       object
ART_V_MANUFACTURER_DESC    object
PPR_ART_CLASS_DESC         object
PPR_N_LEVEL2_ID            int64
dtypes: float64(2), int64(11), object(9)
memory usage: 4.1+ GB


Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let’s make that now.

In [33]:
item_lookup = cleaned_retail[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ART_C_PROD_NUMBER'] = item_lookup.ART_C_PROD_NUMBER.astype(str) # Encode as strings for future lookup ease

In [92]:
item_lookup.head()

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
5,451643,451643.2 Felix Häppchen in Gelee oder Leckerbi...


This can tell us what each item is, such as that ART_C_PROD_NUMBER 3481 is a  Dog Design Decke Hundeschar . Now that this has been created, we need to:

1. Group purchase quantities together by stock code and item ID
2. Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
3. Only include customers with a positive purchase total to eliminate possible errors
4. *Set up our sparse ratings matrix*

This last step is especially important if you don’t want to have unnecessary memory issues! If you think about it, our matrix is going to contain thousands of items and thousands of users with a user/item value required for every possible combination. That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only saving the locations and values of items that are not zero.

The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [65]:
cleaned_retail['CUS_N_KEY_CUSTOMER '] = cleaned_retail.CUS_N_KEY_CUSTOMER.astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['ART_C_PROD_NUMBER', 'SCM_D_QUANTITY', 'CUS_N_KEY_CUSTOMER']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index() # Group together
grouped_cleaned.SCM_D_QUANTITY.loc[grouped_cleaned.SCM_D_QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SCM_D_QUANTITY > 2') # Only get customers where purchase totals were positive

## If we look at our final resulting matrix of grouped purchases, we see the following:

grouped_purchased.head()

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,22,52735,5.0
7,22,397041,4.0
8,22,399251,3.0
12,22,447409,4.0
18,22,543556,3.0


# ToDo: 
Actually here I, perhaps, do not have to filter based on the SCM_D_QUANTITY, but I have to filter the items that have less than 5 interactions, at the same time we have to remove the users that ineracted with less than 5 different items. Check for the remaining items agan, and do it untill the DF is stabilized.


In [66]:
def threshold_likes(df, CUS_N_KEY_CUSTOMER_min, ART_C_PROD_NUMBER_min):
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        ART_C_PROD_NUMBER_counts = df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count()
        df = df[~df.CUS_N_KEY_CUSTOMER.isin(ART_C_PROD_NUMBER_counts[ART_C_PROD_NUMBER_counts < ART_C_PROD_NUMBER_min].index.tolist())]
        CUS_N_KEY_CUSTOMER_counts = df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count()
        df = df[~df.ART_C_PROD_NUMBER.isin(CUS_N_KEY_CUSTOMER_counts[CUS_N_KEY_CUSTOMER_counts < CUS_N_KEY_CUSTOMER_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count().min() >= ART_C_PROD_NUMBER_min)
    assert(df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count().min() >= CUS_N_KEY_CUSTOMER_min)
    
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [67]:
df_lim = threshold_likes(grouped_purchased, 5, 5)

Starting likes info
Number of users: 594238
Number of models: 6869
Sparsity: 0.051%
Ending likes info
Number of users: 137953
Number of models: 5564
Sparsity: 0.163%


In [72]:
customers = list(np.sort(df_lim.CUS_N_KEY_CUSTOMER.unique())) # Get our unique customers
products = list(df_lim.ART_C_PROD_NUMBER.unique()) # Get our unique products that were purchased
quantity = list(df_lim.SCM_D_QUANTITY) # All of our purchases

rows = df_lim.CUS_N_KEY_CUSTOMER.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = df_lim.ART_C_PROD_NUMBER.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [73]:
purchases_sparse

<137953x5564 sparse matrix of type '<class 'numpy.float64'>'
	with 1249663 stored elements in Compressed Sparse Row format>

In [74]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.83719241255042

# Creating a Training and Validation Set

Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set. That normally looks like this:

With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

For this example, because the time period is only 8 months and because of the purchasing type (products), it is most likely products won’t be purchased again in a short time period anyway. This will be a better test. You can see an example here:

pic

Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

As an additional check, we can compare our system to simply recommending the most popular items to every user (beating popularity is a bit difficult). This will be our baseline.

This method of testing isn’t necessarily the “correct” answer, because it depends on how you want to use the recommender system. However, it is a practical way of testing performance I will use for this example.

Now that we have a plan on how to separate our training and testing sets, let’s create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [75]:
import random

In [76]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [77]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

# SLOW Python implementation

# Speeding Up ALS

In [78]:
import implicit

In [79]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

100%|██████████| 50.0/50 [00:19<00:00,  2.76it/s]


# Evaluating the Recommender System

In [80]:
from sklearn import metrics

In [81]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [82]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [83]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.906, 0.869)

We can see that our recommender system beat popularity. Our system had a mean AUC of 0.87, while the popular item benchmark had a lower AUC of 0.869. You can go back and tune the hyperparameters if you wish to see if you can get a higher AUC score. Ideally, you would have separate train, cross-validation, and test sets so that you aren’t overfitting while tuning the hyperparameters, but this setup is adequate to demonstrate how to check that the system is working.

# A Recommendation Example

In [84]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [97]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    return item_lookup.loc[item_lookup.ART_C_PROD_NUMBER.isin(prod_codes)]

We need to look these up by a customer’s ID. Looking at the list of customers:

In [98]:
customers_arr[:5]

array([ 22,  84, 205, 208, 221], dtype=int64)

we can see that the first customer listed has an ID of 12346. Let’s examine their purchases from the training set.

In [103]:
#--------------- missed a part

In [104]:
from sklearn.preprocessing import MinMaxScaler

In [120]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        print(code)
        rec_list.append([code, item_lookup.ART_V_ART_DESCRIPTION.loc[item_lookup.ART_C_PROD_NUMBER == str(code)].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    print (descriptions)
    #final_frame = pd.DataFrame({'ART_C_PROD_NUMBER': codes, 'ART_V_ART_DESCRIPTION': descriptions}) # Create a dataframe 
    #return final_frame[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']] # Switch order of columns around
    return 0

In [121]:
rec_items(208, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

508482
531249
471676
508454
206642
479505
556340
592201
592299
597294
['508482.1 Royal Canin Veterinary Diet Feline Renal - Huhn 12 x 85 g', '531249.2 Kattovit Niere/Renal Pouch 6 x 85 g - 6 x 85 g Truthahn', '471676.0 Royal Canin Veterinary Diet Feline Renal mit Thunfisch - 12 x 85 g', '508454.2 Royal Canin Veterinary Diet Feline Sensitivity Control Huhn - 48 x 100 g', "206642.4 Hill's Prescription Diet k/d Kidney Care Katzenfutter - mit Huhn 12 x 85 g", '479505.2 Kattovit Gastro Pouch 6 x 85 g - 6 x 85 g Ente & Reis', '556340.10 Sparpaket Kattovit Pouches 24 x 85 g - Renal mit Rind', '592201.4 Animonda Integra Protect Adult Niere 24 x 100 g Schale - Mix (4 Sorten)', '592299.3 Animonda Integra Protect Adult Niere Schale 6 x 100 g - mit Rind', '597294.3 10 + 2 gratis! 12 x 100 g Animonda Integra Protect Adult - Sensitive Huhn Pur']


0

In [114]:
item_lookup[item_lookup['ART_C_PROD_NUMBER']=='174338']

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
229,174338,174338.9 Megapack Gourmet Perle 24 x 85 g - Er...
230,174338,174338.10 Megapack Gourmet Perle 24 x 85 g - E...
231,174338,174338.12 Megapack Gourmet Perle 24 x 85 g - D...
353,174338,174338.2 Megapack Gourmet Perle 24 x 85 g - Er...
358,174338,174338.11 Megapack Gourmet Perle 24 x 85 g - ...
685,174338,174338.6 Megapack Gourmet Perle 24 x 85 g - Du...
998,174338,174338.5 Megapack Gourmet Perle 24 x 85 g - Er...
1659,174338,174338.3 Megapack Gourmet Perle 24 x 85 g - Er...
1662,174338,174338.7 Megapack Gourmet Perle 24 x 85 g - Du...
3610,174338,174338.0 Megapack Gourmet Perle 24 x 85 g - Er...


In [122]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 208]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
237162,354619255,83276391,0,01-MAR-17,12937,15976,208,1.0,9390,7146,...,7546,7546.4 Yarrah Bio Chunks 6 x 100 g - Huhn & Ri...,7546.4 Yarrah Bio Chunks 6 x 100 g - Huhn & Ri...,Katzenfutter nass,Yarrah Biofutter,Schalen,Schalen,Yarrah,Canned Food Cat / Dog,3
315932,354619254,83276391,0,01-MAR-17,1760932,93213424,208,1.0,67405,48484,...,508447,508447.2 Royal Canin Veterinary Diet Feline Ga...,508447.2 Royal Canin Veterinary Diet Feline Ga...,Katzenfutter nass,Royal Canin Veterinary Diet,Magen & Darm,Magen & Darm,Royal Canin,NaN,3
2079403,347158965,81578636,0,28-JAN-17,1935568,14027,208,1.0,20342,16373,...,565817,565817.3 Probierpreis: 5 l Cat's Best Öko Plus...,565817.3 Probierpreis: 5 l Cat's Best Öko Plus...,Katzenstreu,Cat's Best,Probiergröße,Probiergröße,JRS - Rettenmaier,cat litter,3
2079404,347158964,81578636,0,28-JAN-17,1703927,93213424,208,1.0,67405,48484,...,508447,508447.1 Royal Canin Veterinary Diet Feline Ga...,508447.1 Royal Canin Veterinary Diet Feline Ga...,Katzenfutter nass,Royal Canin Veterinary Diet,Magen & Darm,Magen & Darm,Royal Canin,NaN,3
2079405,347158962,81578636,0,28-JAN-17,1950770,104015560,208,1.0,69109,49235,...,354551,354551.3 Kratztonne Natural Paradise Standard ...,354551.3 Kratztonne Natural Paradise Standard ...,Kratzbaum & Kratzmöbel,Kratztonnen,Natural Paradise Kratztonne,Natural Paradise Kratztonne,GeoP,bulky / big cat / dog acc.,3
2812306,350244102,82285828,0,10-FEB-17,531067,8113029,208,1.0,32198,3187,...,3187,3187.4 Cat's Best Original Katzenstreu - 20 l ...,3187.4 Cat's Best Original Katzenstreu - 20 l ...,Katzenstreu,Cat's Best,Cat's Best,Cat's Best,JRS - Rettenmaier,cat litter,3
2812307,350244100,82285828,0,10-FEB-17,12933,15978,208,1.0,1451,7146,...,7546,7546.0 Yarrah Bio Chunks 6 x 100 g - Fisch mit...,7546.0 Yarrah Bio Chunks 6 x 100 g - Fisch mit...,Katzenfutter nass,Yarrah Biofutter,Schalen,Schalen,Yarrah,Canned Food Cat / Dog,3
2812308,350244101,82285828,0,10-FEB-17,1760932,93213424,208,1.0,67405,48484,...,508447,508447.2 Royal Canin Veterinary Diet Feline Ga...,508447.2 Royal Canin Veterinary Diet Feline Ga...,Katzenfutter nass,Royal Canin Veterinary Diet,Magen & Darm,Magen & Darm,Royal Canin,NaN,3
4929774,358216219,84105975,0,16-MAR-17,1760932,93213424,208,1.0,67405,48484,...,508447,508447.2 Royal Canin Veterinary Diet Feline Ga...,508447.2 Royal Canin Veterinary Diet Feline Ga...,Katzenfutter nass,Royal Canin Veterinary Diet,Magen & Darm,Magen & Darm,Royal Canin,NaN,3
4929918,358216218,84105975,0,16-MAR-17,1704469,78428032,208,1.0,65112,46859,...,3187,3187.10 Cat's Best Original Katzenstreu - pass...,3187.10 Cat's Best Original Katzenstreu - pass...,Katzenstreu,Cat's Best,Cat's Best,Cat's Best,Zhejiang Zhongda Sunjoy,"small cat / dog acc., container and local merc...",3
